## Getting Started

If you've ever written code before, the problems caused by the code below will not be new to you, but it is worth making explicit.
The code mimics a simple data pipeline, which makes a call to an API service, augments the data, and then writes the results to our database.

The major difference is that the API call that we are making will fail half of the time. This is hopefully much more frequently than your API calls will fail in production, but it is useful for demonstration purposes.

In [10]:
import random
from prefect import flow, task

In [2]:

def call_unreliable_api():  # sourcery skip: raise-specific-error
    choices = [{"data": 42}, "failure"]
    res = random.choice(choices)
    if res == "failure":
        raise Exception("Our unreliable service failed")
    else:
        return res


def augment_data(data: dict, msg: str):
    data["message"] = msg
    return data


def write_results_to_database(data: dict):
    print(f"Wrote {data} to database successfully!")
    return "Success!"


def pipeline(msg: str):
    api_result = call_unreliable_api()
    augmented_data = augment_data(data=api_result, msg=msg)
    write_results_to_database(augmented_data)

In [5]:
for _ in range(10):
    pipeline(msg="Super Special Message")

Wrote {'data': 42, 'message': 'Super Special Message'} to database successfully!
Wrote {'data': 42, 'message': 'Super Special Message'} to database successfully!


Exception: Our unreliable service failed

### Negative Engineering

This is obviously a trivial example, and as engineers, we know to expect these things and deal with them. But, dealing with ways code fails is NOT what we set out to do. We set out to write a data pipeline.

The process of writing code that deals with failures, instead of writing code that performs the actions that we want to be done, is something that we at Prefect refer to as *Negative Engineering*.

Negative Engineering happens when engineers write defensive code to make sure the positive code actually runs. It must anticipate the almost limitless number of ways that code can fail, and is a massive time sink.

Prefect aims to eliminate as much negative engineering as possible for you.


### Using a Prefect Flow

It's easier to show than it is to tell, so let's run this next block and then we'll explain what is happening.

#### Creating a flow

To create a flow, we simply import flow from prefect and then add it as a decorator to our pipeline function. You can see the modifications that we’ve made to our flow below. Any lines that have modifications will be tagged with the comment # NEW **** .

In [7]:
def call_unreliable_api():  # sourcery skip: raise-specific-error
    choices = [{"data": 42}, "failure"]
    res = random.choice(choices)
    if res == "failure":
        raise Exception("Our unreliable service failed")
    else:
        return res


def augment_data(data: dict, msg: str):
    data["message"] = msg
    return data


def write_results_to_database(data: dict):
    print(f"Wrote {data} to database successfully!")
    return "Success!"

@flow   # NEW ****
def pipeline(msg: str):
    api_result = call_unreliable_api()
    augmented_data = augment_data(data=api_result, msg=msg)
    write_results_to_database(augmented_data)

In [8]:
pipeline("Trying a flow!")

20:48:28.060 | INFO    | prefect.engine - Created flow run 'thundering-goshawk' for flow 'pipeline'
20:48:28.060 | INFO    | Flow run 'thundering-goshawk' - Using task runner 'ConcurrentTaskRunner'
20:48:28.069 | WARNING | Flow run 'thundering-goshawk' - No default storage is configured on the server. Results from this flow run will be stored in a temporary directory in its runtime environment.
20:48:28.099 | ERROR   | Flow run 'thundering-goshawk' - Encountered exception during execution:
Traceback (most recent call last):
  File "/Users/mjboothaus/try-prefect2/.venv/lib/python3.9/site-packages/prefect/engine.py", line 520, in orchestrate_flow_run
    result = await run_sync_in_interruptible_worker_thread(
  File "/Users/mjboothaus/try-prefect2/.venv/lib/python3.9/site-packages/prefect/utilities/asyncio.py", line 116, in run_sync_in_interruptible_worker_thread
    tg.start_soon(
  File "/Users/mjboothaus/try-prefect2/.venv/lib/python3.9/site-packages/anyio/_backends/_asyncio.py", line

Failed(message='Flow run encountered an exception.', type=FAILED, result=Exception('Our unreliable service failed'))

In [9]:
!ls ~/.prefect

auth.toml    config.toml  orion.db     orion.db-wal
backend.toml flows        orion.db-shm results


#### Making our flows better with tasks

Flows are only the first step in orchestrating our data pipelines. The next step is adding Prefect task.

A task can be thought of as a discrete unit of work. In practice, you'll often simply convert the functions that make up your flow into tasks.

Like flows, tasks are created by adding a decorator. We'll demonstrate below.

In [11]:
 @task   # NEW ****
def call_unreliable_api():
    choices = [{"data": 42}, "failure"]
    res = random.choice(choices)
    if res == "failure":
        raise Exception("Our unreliable service failed")
    else:
        return res

@task   # NEW ****
def augment_data(data: dict, msg: str):
    data["message"] = msg
    return data

@task   # NEW ****
def write_results_to_database(data: dict):
    print(f"Wrote {data} to database successfully!")
    return "Success!"

@flow 
def pipeline(msg: str):
    api_result = call_unreliable_api()
    augmented_data = augment_data(data=api_result, msg=msg)
    write_results_to_database(augmented_data)

/Users/mjboothaus/try-prefect2/.venv/lib/python3.9/site-packages/prefect/flows.py:173: UserWarning: A flow named 'pipeline' and defined at '/var/folders/wc/zhl433ms1fx5rvtvcrkw6dgw0000gn/T/ipykernel_62685/4044207864.py:20' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_unique_name', ...)`
  warnings.warn(


In [17]:
for i in range(5):
    print(f"Run #{i}")
    pipeline("Trying a flow with tasks!")

21:11:53.251 | INFO    | prefect.engine - Created flow run 'asparagus-albatross' for flow 'pipeline'
21:11:53.251 | INFO    | Flow run 'asparagus-albatross' - Using task runner 'ConcurrentTaskRunner'
21:11:53.258 | WARNING | Flow run 'asparagus-albatross' - No default storage is configured on the server. Results from this flow run will be stored in a temporary directory in its runtime environment.
21:11:53.326 | INFO    | Flow run 'asparagus-albatross' - Created task run 'call_unreliable_api-466f2784-0' for task 'call_unreliable_api'
21:11:53.344 | INFO    | Flow run 'asparagus-albatross' - Created task run 'augment_data-960bb844-0' for task 'augment_data'
21:11:53.353 | ERROR   | Task run 'call_unreliable_api-466f2784-0' - Encountered exception during execution:
Traceback (most recent call last):
  File "/Users/mjboothaus/try-prefect2/.venv/lib/python3.9/site-packages/prefect/engine.py", line 890, in orchestrate_task_run
    result = await run_sync_in_interruptible_worker_thread(
  Fi

Run #0


21:11:53.378 | ERROR   | Task run 'call_unreliable_api-466f2784-0' - Finished in state Failed('Task run encountered an exception.')
21:11:53.415 | ERROR   | Flow run 'asparagus-albatross' - Finished in state Failed('1/3 states failed.')
21:11:53.492 | INFO    | prefect.engine - Created flow run 'ivory-chinchilla' for flow 'pipeline'
21:11:53.492 | INFO    | Flow run 'ivory-chinchilla' - Using task runner 'ConcurrentTaskRunner'
21:11:53.498 | WARNING | Flow run 'ivory-chinchilla' - No default storage is configured on the server. Results from this flow run will be stored in a temporary directory in its runtime environment.
21:11:53.529 | INFO    | Flow run 'ivory-chinchilla' - Created task run 'call_unreliable_api-466f2784-0' for task 'call_unreliable_api'
21:11:53.545 | INFO    | Flow run 'ivory-chinchilla' - Created task run 'augment_data-960bb844-0' for task 'augment_data'
21:11:53.565 | INFO    | Flow run 'ivory-chinchilla' - Created task run 'write_results_to_database-fbbf5571-0' fo

Run #1
Wrote {'data': 42, 'message': 'Trying a flow with tasks!'} to database successfully!


21:11:53.641 | INFO    | Task run 'write_results_to_database-fbbf5571-0' - Finished in state Completed()
21:11:53.665 | INFO    | Flow run 'ivory-chinchilla' - Finished in state Completed('All states completed.')
21:11:53.751 | INFO    | prefect.engine - Created flow run 'inventive-pig' for flow 'pipeline'
21:11:53.751 | INFO    | Flow run 'inventive-pig' - Using task runner 'ConcurrentTaskRunner'
21:11:53.757 | WARNING | Flow run 'inventive-pig' - No default storage is configured on the server. Results from this flow run will be stored in a temporary directory in its runtime environment.
21:11:53.788 | INFO    | Flow run 'inventive-pig' - Created task run 'call_unreliable_api-466f2784-0' for task 'call_unreliable_api'
21:11:53.803 | INFO    | Flow run 'inventive-pig' - Created task run 'augment_data-960bb844-0' for task 'augment_data'
21:11:53.824 | INFO    | Flow run 'inventive-pig' - Created task run 'write_results_to_database-fbbf5571-0' for task 'write_results_to_database'
21:11:5

Run #2
Wrote {'data': 42, 'message': 'Trying a flow with tasks!'} to database successfully!


21:11:53.895 | INFO    | Flow run 'inventive-pig' - Finished in state Completed('All states completed.')
21:11:53.974 | INFO    | prefect.engine - Created flow run 'meek-ammonite' for flow 'pipeline'
21:11:53.975 | INFO    | Flow run 'meek-ammonite' - Using task runner 'ConcurrentTaskRunner'
21:11:53.981 | WARNING | Flow run 'meek-ammonite' - No default storage is configured on the server. Results from this flow run will be stored in a temporary directory in its runtime environment.
21:11:54.011 | INFO    | Flow run 'meek-ammonite' - Created task run 'call_unreliable_api-466f2784-0' for task 'call_unreliable_api'
21:11:54.027 | INFO    | Flow run 'meek-ammonite' - Created task run 'augment_data-960bb844-0' for task 'augment_data'
21:11:54.037 | ERROR   | Task run 'call_unreliable_api-466f2784-0' - Encountered exception during execution:
Traceback (most recent call last):
  File "/Users/mjboothaus/try-prefect2/.venv/lib/python3.9/site-packages/prefect/engine.py", line 890, in orchestrat

Run #3
Run #4


21:11:54.171 | INFO    | prefect.engine - Created flow run 'hairy-barracuda' for flow 'pipeline'
21:11:54.171 | INFO    | Flow run 'hairy-barracuda' - Using task runner 'ConcurrentTaskRunner'
21:11:54.177 | WARNING | Flow run 'hairy-barracuda' - No default storage is configured on the server. Results from this flow run will be stored in a temporary directory in its runtime environment.
21:11:54.210 | INFO    | Flow run 'hairy-barracuda' - Created task run 'call_unreliable_api-466f2784-0' for task 'call_unreliable_api'
21:11:54.227 | INFO    | Flow run 'hairy-barracuda' - Created task run 'augment_data-960bb844-0' for task 'augment_data'
21:11:54.247 | INFO    | Flow run 'hairy-barracuda' - Created task run 'write_results_to_database-fbbf5571-0' for task 'write_results_to_database'
21:11:54.258 | INFO    | Task run 'call_unreliable_api-466f2784-0' - Finished in state Completed()
21:11:54.285 | INFO    | Task run 'augment_data-960bb844-0' - Finished in state Completed()
21:11:54.313 | IN

Wrote {'data': 42, 'message': 'Trying a flow with tasks!'} to database successfully!


#### Adding retries

The next feature that we will demo is the ability to retry a task. We know that tasks will inevitably fail. Sometimes this requires complex behavior, but other times we simply need to try again after a brief delay. We can do this with the `retries` and `retry_delay_seconds` parameters.

This will be helpful for our unreliable API call.

In [18]:
@task(name="Get data from API", retries=4, retry_delay_seconds=3)
def call_unreliable_api():
    choices = [{"data": 42}, "failure"]
    res = random.choice(choices)
    if res == "failure":
        raise Exception("Our unreliable service failed")
    else:
        return res

@task   # NEW ****
def augment_data(data: dict, msg: str):
    data["message"] = msg
    return data

@task   # NEW ****
def write_results_to_database(data: dict):
    print(f"Wrote {data} to database successfully!")
    return "Success!"

@flow 
def pipeline(msg: str):
    api_result = call_unreliable_api()
    augmented_data = augment_data(data=api_result, msg=msg)
    write_results_to_database(augmented_data)

/Users/mjboothaus/try-prefect2/.venv/lib/python3.9/site-packages/prefect/tasks.py:165: UserWarning: A task named 'augment_data' and defined at '/var/folders/wc/zhl433ms1fx5rvtvcrkw6dgw0000gn/T/ipykernel_62685/155245748.py:10' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/Users/mjboothaus/try-prefect2/.venv/lib/python3.9/site-packages/prefect/tasks.py:165: UserWarning: A task named 'write_results_to_database' and defined at '/var/folders/wc/zhl433ms1fx5rvtvcrkw6dgw0000gn/T/ipykernel_62685/155245748.py:15' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/Users/mjboothaus/try-prefect2/.venv/lib/python3.9/site-packages/prefect/flows.py:173: UserWarning: A flow named 'pipeline' and defined at '/var/folders/wc/zhl433ms1fx5rvtvcrkw6dgw0000gn/T/ipykernel_62685/155245748.py:20' confl

In [20]:
pipeline("Trying a flow with tasks and retries!")

21:17:29.044 | INFO    | prefect.engine - Created flow run 'elite-barnacle' for flow 'pipeline'
21:17:29.046 | INFO    | Flow run 'elite-barnacle' - Using task runner 'ConcurrentTaskRunner'
21:17:29.060 | WARNING | Flow run 'elite-barnacle' - No default storage is configured on the server. Results from this flow run will be stored in a temporary directory in its runtime environment.
21:17:29.104 | INFO    | Flow run 'elite-barnacle' - Created task run 'Get data from API-466f2784-0' for task 'Get data from API'
21:17:29.125 | INFO    | Flow run 'elite-barnacle' - Created task run 'augment_data-960bb844-0' for task 'augment_data'
21:17:29.143 | ERROR   | Task run 'Get data from API-466f2784-0' - Encountered exception during execution:
Traceback (most recent call last):
  File "/Users/mjboothaus/try-prefect2/.venv/lib/python3.9/site-packages/prefect/engine.py", line 890, in orchestrate_task_run
    result = await run_sync_in_interruptible_worker_thread(
  File "/Users/mjboothaus/try-prefe

Wrote {'data': 42, 'message': 'Trying a flow with tasks and retries!'} to database successfully!


Completed(message='All states completed.', type=COMPLETED, result=[Completed(message=None, type=COMPLETED, result={'data': 42, 'message': 'Trying a flow with tasks and retries!'}), Completed(message=None, type=COMPLETED, result={'data': 42, 'message': 'Trying a flow with tasks and retries!'}), Completed(message=None, type=COMPLETED, result='Success!')])